In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu Sep  5 14:17:02 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.993
BogoMIPS:              5851.93
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [2]:
%env wd=/home/sam/analyses/20190905_pgen_v074.a3_repeats_counts
wd="/home/sam/analyses/20190905_pgen_v074.a3_repeats_counts"

%env rysnc_owl=owl:/volume1/web/halfshell/genomic-databank/
%env gff=Panopea-generosa-vv0.74.a3.TE.gff3
%env wget_gffs=--directory-prefix=${wd} --recursive --quiety --no-directories --no-check-certificate --no-parent --accept 'Panopea-generosa-vv0.74.a3.TE.gff3' https://owl.fish.washington.edu/halfshell/genomic-databank/

# Set genome size to 942Mbp
GENOME_SIZE = 942000000


env: wd=/home/sam/analyses/20190905_pgen_v074.a3_repeats_counts
env: rysnc_owl=owl:/volume1/web/halfshell/genomic-databank/
env: gff=Panopea-generosa-vv0.74.a3.TE.gff3
env: wget_gffs=--directory-prefix=$/home/sam/analyses/20190905_pgen_v074.a3_repeats_counts --recursive --quiety --no-directories --no-check-certificate --no-parent --accept 'Panopea-generosa-vv0.74.a3.TE.gff3' https://owl.fish.washington.edu/halfshell/genomic-databank/


In [22]:
# Calculate percentage of genome comprised of a given feature
def ind_repeats_percent(feature_length_sum): 
    return round(float(feature_length_sum / GENOME_SIZE * 100), 2)

### Import Python modules

In [4]:
import fnmatch
import os
import pandas

#### Create necessary directories

In [5]:
%%bash
mkdir --parents ${wd}

In [6]:
cd {wd}

/home/sam/analyses/20190905_pgen_v074.a3_repeats_counts


#### Download _Panopea generosa_ GFFs for v074.a3.

Info on GFFs is here: [https://github.com/RobertsLab/resources/wiki/Genomic-Resources#genome-feature-tracks-3)

In [12]:
%%bash

rsync \
--archive \
--verbose \
--progress \
--include="${gff}" \
--exclude="*" \
"${rysnc_owl}" \
.

echo ""
echo ""
echo "----------------------------------------------------------"

ls -lh

receiving incremental file list
./
Panopea-generosa-vv0.74.a3.TE.gff3
    160,525,914 100%  986.11kB/s    0:02:38 (xfr#1, to-chk=0/2)

sent 80 bytes  received 160,545,654 bytes  564,308.38 bytes/sec
total size is 160,525,914  speedup is 1.00


----------------------------------------------------------
total 154M
-rw-rw-r-- 1 sam users 154M Sep  5 08:48 Panopea-generosa-vv0.74.a3.TE.gff3


#### If need to download via wget, uncomment lines in the cell below

In [13]:
# %%bash
# time \
# wget "${wget_gffs}"

# ls -lh ${wd}

In [14]:
%%bash
head ${gff}

##gff-version 3
##Generated using GenSAS, Tuesday 9th of July 2019 09:21:16 PM
##Project Name : Pgenerosa_v074
##Job Name  : RepeatMasker
##Tool      : RepeatMasker 4.0.7
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	1876	1934	12	+	.	ID=19534.GS22252505.PGEN_.repeat00000002;Name=19534.GS22252505.PGEN_.repeat00000002;repeat_match=%28ATTC%29n;repeat_class=Simple_repeat;
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	2962	3024	249	-	.	ID=19534.GS22252505.PGEN_.repeat00000003;Name=19534.GS22252505.PGEN_.repeat00000003;repeat_match=HalSINE1;repeat_class=Unspecified;
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	8054	8084	28	+	.	ID=19534.GS22252505.PGEN_.repeat00000004;Name=19534.GS22252505.PGEN_.repeat00000004;repeat_match=%28CGT%29n;repeat_class=Simple_repeat;
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	8318	861

In [17]:
%%bash

# Set array of GFF features to parse out
gff_features=(DNA LINE LTR SINE Simple_repeat Unknown)

# Loop through array and create new GFFs from each feature
for feature in ${gff_features[@]}
do
    echo "Parsing ${feature} from ${gff}..."
    echo "Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3"
    head -n 5 ${gff} > Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3
    echo "Parsing matching feature lines for ${feature} feature..."
    grep ${feature} ${gff} >> Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3
    echo "Done with parsing ${feature}."
    feature_count=$(tail --lines +6 Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3 | wc -l)
    echo "Identified ${feature_count} ${feature} features."
    echo "Output file is: Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3"
    echo ""
done

echo "----------------------------------------------"
echo ""
ls -lh
  

Parsing DNA from Panopea-generosa-vv0.74.a3.TE.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.DNA.gff3
Parsing matching feature lines for DNA feature...
Done with parsing DNA.
Identified 38612 DNA features.
Output file is: Panopea-generosa-vv0.74.a3.repeats.DNA.gff3

Parsing LINE from Panopea-generosa-vv0.74.a3.TE.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.LINE.gff3
Parsing matching feature lines for LINE feature...
Done with parsing LINE.
Identified 73797 LINE features.
Output file is: Panopea-generosa-vv0.74.a3.repeats.LINE.gff3

Parsing LTR from Panopea-generosa-vv0.74.a3.TE.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.LTR.gff3
Parsing matching feature lines for LTR feature...
Done with parsing LTR.
Identified 11752 LTR features.
Output file is: Panopea-generosa-vv0.74.a3.repeats.LTR.gff3

Parsing SINE from Panopea-generosa-vv0.74.a3.TE.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.SINE.gff3
Parsing m

### Check the output files

In [19]:
%%bash
# Check the output files
for file in Panopea-generosa-vv0.74.a3.repeats*.gff3
do
    echo ""
    echo ""
    echo "${file}"
    echo "----------------------------------------------"
    head ${file}
done



Panopea-generosa-vv0.74.a3.repeats.DNA.gff3
----------------------------------------------
##gff-version 3
##Generated using GenSAS, Monday 15th of July 2019 06:30:16 AM
##Project Name : Pgenerosa_v074
##Job Name  : Masked Repeat Consensus
##Tool      : Mask Sequence Consensus
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	60243	60322	325	-	.	ID=19647.GS22252505.PGEN_.repeat00000029;Name=19534.GS22252505.PGEN_.repeat00000030;repeat_match=DNA2-25_CGi;repeat_class=Unspecified;
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	99944	100057	231	-	.	ID=19647.GS22252505.PGEN_.repeat00000070;Name=19534.GS22252505.PGEN_.repeat00000071;repeat_match=DNA-8-3_HM;repeat_class=Unspecified;
PGA_scaffold1__77_contigs__length_89643857	GenSAS_5d250896def4c-repeatmasker	repeat_region	99996	100093	243	-	.	ID=19647.GS22252505.PGEN_.repeat00000072;Name=19534.GS22252505.PGEN_.repeat00000073;repeat_match=DNA-8-3_HM;repeat_c

In [20]:
# Set list of column header names
gff_header = ['seqid','source','type','start','end','score','strand','phase','attributes']

### Get sequence length stats for repeat features

In [23]:
total_repeats_percent = 0

for file in os.listdir('.'):
    if fnmatch.fnmatch(file, 'Panopea-generosa-vv0.74.a3.repeats*.gff3'):
        print('\n' * 2)
        print(file)
        print("-------------------------")
        # Import GFF.
        # Skip first row and file is tab-separated
        gff=pandas.read_csv(file, header=None, skiprows=5, sep="\t")
        # Rename columns
        gff.columns = gff_header
        # Subtract start value from end value.
        # Have to add 1 so that sequence length can't equal zero
        gff['seqlength'] = gff.apply(lambda position: position['end'] - position['start'] + 1, axis=1)
        gff_sum = gff['seqlength'].sum()
        
        total_repeats_percent += ind_repeats_percent(gff_sum)
        print ("percent" , ind_repeats_percent(gff_sum))
        
        # Apply functions in list to seqlength column
        gff_stats = gff['seqlength'].agg(['sum', 'mean', 'min', 'median', 'max'])
        
        print (gff_stats.round(2))
print('\n' * 2) 
print("-------------------------")
print ("Repeats composition of genome (percent):" , total_repeats_percent)




Panopea-generosa-vv0.74.a3.repeats.DNA.gff3
-------------------------
percent 1.2
sum       11316890.00
mean           293.09
min              1.00
median         154.00
max           7012.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a3.repeats.Simple_repeat.gff3
-------------------------


/home/sam/programs/minicocnda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


percent 2.03
sum       19132744.00
mean            63.78
min              1.00
median          41.00
max          12422.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a3.repeats.LINE.gff3
-------------------------
percent 3.11
sum       29258694.00
mean           396.48
min             11.00
median         227.00
max           6604.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a3.repeats.LTR.gff3
-------------------------
percent 0.46
sum       4355629.00
mean          370.63
min             1.00
median        276.00
max          6541.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a3.repeats.SINE.gff3
-------------------------
percent 2.3
sum       21645991.00
mean           147.84
min              1.00
median         142.00
max            934.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a3.repeats.Unknown.gff3
-------------------------
percent 31.14
sum       2.933161e+08
mean      2.001500e+02
min       1.100000e+01
media